## 2 Model - DONE

The space reentry model to be used is a very simplified one, only considering the air resistance force seen in the classes, and also some of the Newton laws, including the gravitational Newton law.
The air resistance empiric law to be used is:

$$ F_d = -\frac{1}{2} \rho A C_d v^2 $$

with $\rho$ the air density, $A$ the surface considered for the drag coefficient, $C_d$ the drag coefficient that depends 
on the geometry of the body (it also has some dependence on the velocity but we will consider it constant) and $v$ the velocity.


In [ ]:
def air_resistance_force(air_density, surface_area, drag_coefficient, velocity):
    """
    Calculate the air resistance force based on the given parameters.

    Parameters:
    air_density (float): The air density (ρ) in kg/m^3
    surface_area (float): The surface area (A) considered for the drag coefficient in m^2
    drag_coefficient (float): The drag coefficient (Cd)
    velocity (float): The velocity (v) in m/s

    Returns:
    float: The air resistance force (Fd) in Newtons (N)
    """
    force = -0.5 * air_density * surface_area * drag_coefficient * velocity**2
    return force


# Example values
air_density = 1.225  # kg/m^3, typical air density at sea level
surface_area = 2.0  # m^2, example surface area
drag_coefficient = 1.0  # example drag coefficient
velocity = 30.0  # m/s, example velocity

# Calculate the air resistance force
fd = air_resistance_force(air_density, surface_area, drag_coefficient, velocity)

print(f"Air Resistance Force: {fd} N")

# Output should be a negative value representing the force acting opposite to the direction of motion

### 3.1 g values

The g values are the total acceleration the vessel and crew are subjected to, less the gravity acceleration and usually measured in g, i.e. multiples of the gravity acceleration on the sea level. Don’t forget that the gravity acceleration is only in one direction while the total acceleration can have components on the vertical, like gravity, and on horizontal direction. For this measurement the students can use for the standard gravity acceleration at sea level the value of 10 m·s⁻².


In [ ]:
import math

# Constants
g = 10.0  # standard gravity acceleration at sea level in m/s^2

# Example values for vertical and horizontal accelerations
vertical_acceleration = 15.0  # in m/s^2
horizontal_acceleration = 20.0  # in m/s^2


# Function to calculate total acceleration and g values
def calculate_g_value(vertical_acceleration, horizontal_acceleration):
    # Calculate total acceleration
    total_acceleration = math.sqrt(
        vertical_acceleration**2 + horizontal_acceleration**2
    )

    # Calculate g value
    g_value = total_acceleration / g

    return total_acceleration, g_value


# Calculate the total acceleration and g values
total_acceleration, g_value = calculate_g_value(
    vertical_acceleration, horizontal_acceleration
)

print(f"Total Acceleration: {total_acceleration} m/s^2")
print(f"g Value: {g_value} g")

### 3.2 Horizontal distance projected on the Earth surface

This value pretends to measure the distance from the point on the Earth surface vertically below the crossing the reentry interface by the space module to the point it touches down. We can not use the x values of the trajectory because that movement is compose of arc of circles at different heights from the Earth center, which would increase the values obtained.

An approximation would be to consider that at each step we consider as if each step in the forward method was taken at constant height (which is sometimes a rough approximation) and compute the θ angle measured from the center of the Earth that step movement had made. Considering it at constant height and equal to the initial height of the step, we would have:

$$\theta_i = \frac{x_i - x_{i-1}}{R_{\text{Earth}} + y_i}$$

The sum of all θ<sub>i</sub> will give the total angle from interface reentry crossing to touchdown as measured from the center of the Earth, and now it is possible to compute the distance as measured at the Earth surface by:

$$\text{distance} = R_{\text{Earth}} \times \theta $$


In [ ]:
def calculate_horizontal_distance(x, y, R_earth=6371):
    """
    Calculate the horizontal distance projected on the Earth's surface.

    Parameters:
    x (list or np.array): x-coordinates
    y (list or np.array): y-coordinates
    R_earth (float): Radius of the Earth in kilometers. Default is 6371 km.

    Returns:
    float: Horizontal distance on the Earth's surface in kilometers.
    """
    theta = 0
    n = len(x)

    for i in range(1, n):
        theta_i = (x[i] - x[i - 1]) / (R_earth + y[i])
        theta += theta_i

    distance = R_earth * theta
    return distance


# Example usage:
x = [0, 1, 2, 3]  # Example x-coordinates (in km or any consistent unit)
y = [100, 101, 102, 103]  # Example y-coordinates (in km or any consistent unit)

distance = calculate_horizontal_distance(x, y)

print(f"Horizontal distance projected on the Earth's surface: {distance} km")

In [ ]:
import math

v0 = 15000 # Initial velocity in m/s
alpha = 15 # downward angle with the horizontal in degrees

# Convert angle to radians
alphaRad = math.radians(alpha)

v0x = v0 * math.cos(alphaRad) # Initial horizontal velocity
v0y = v0 * math.sin(alphaRad) # Initial vertical velocity

print(f"Initial horizontal velocity: {v0x} m/s")
print(f"Initial vertical velocity: {v0y} m/s")

In [ ]:
from scipy.optimize import fsolve

# Initial conditions
x = 0.0
y = 100000.0
vx = 5000.0
vy = -1000.0

positions_backward = [(x, y)]
velocities_backward = [(vx, vy)]

def backward_euler(vx, vy, x, y, dt):
    def equations(vars):
        vx_next, vy_next = vars
        v_next = np.sqrt(vx_next**2 + vy_next**2)
        Fd_next = drag_force(v_next, y)
        ax_next = -Fd_next * vx_next / (m * v_next)
        ay_next = -g - (Fd_next * vy_next / (m * v_next))
        
        eq1 = vx_next - vx - ax_next * dt
        eq2 = vy_next - vy - ay_next * dt
        return [eq1, eq2]

    vx_next, vy_next = fsolve(equations, (vx, vy))
    x_next = x + vx_next * dt
    y_next = y + vy_next * dt
    return x_next, y_next, vx_next, vy_next

# Simulation loop
for _ in range(int(5000 / dt)):
    x, y, vx, vy = backward_euler(vx, vy, x, y, dt)
    positions_backward.append((x, y))
    velocities_backward.append((vx, vy))
    
    if y <= 0:
        break

# Output results
print("Final position (backward):", positions_backward[-1])
print("Final velocity (backward):", velocities_backward[-1])
